In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Kütüphaneleri İndirme

In [ ]:
pip install requests

In [ ]:
pip install BeautifulSoup4

# Veriyi Çekme

### Filmlerin listelendiği sayfadan film linklerinin alınması

In [2]:
url = "https://www.imdb.com/chart/top/"
response = requests.get(url)
html = response.content
parsed = BeautifulSoup(html,"html.parser")
filmler_head = parsed.find("tbody",{"class":"lister-list"})
filmler = filmler_head.find_all("tr")
print('Toplam film sayısı : ' + str(len(filmler)))

linkler = []

for film in filmler:
    url = film.find("td",{"class":"titleColumn"}).find('a')
    linkler.append(url['href'])

Toplam film sayısı : 250


### Tek tek filmlere gidilip gerekli bilgilerin çekilmesi ve listeye eklenmesi

In [34]:
filmler = []

for link in linkler:
    response = requests.get("https://www.imdb.com" + link)
    html = response.content
    parsed = BeautifulSoup(html,"html.parser")
    
    isim = parsed.find("div",{"class":"TitleBlock__TitleContainer-sc-1nlhx7j-1 jxsVNt"}).find("h1").text
    
    yapimYili = parsed.find_all("a",{"class":"ipc-link ipc-link--baseAlt ipc-link--inherit-color TitleBlockMetaData__StyledTextLink-sc-12ein40-1 rgaOW"})
    if len(yapimYili) >= 1 : 
        yapimYili = yapimYili[0].text 
    else:
        yapimYili = ""
    
    yasSiniri = parsed.find_all("span",{"class":"TitleBlockMetaData__ListItemText-sc-12ein40-2 jedhex"})
    if len(yasSiniri) >= 2 : 
        yasSiniri = yasSiniri[1].text 
    else:
        yasSiniri = ""
        
    sure = parsed.find_all("li",{"class":"ipc-inline-list__item"})
    if len(sure) >= 3 : 
        sure = sure[2].text 
    else:
        sure = ""
    
    puan = parsed.find("span",{"class":"AggregateRatingButton__RatingScore-sc-1ll29m0-1 iTLWoV"}).text
    
    yonetmen = parsed.find("a",{"class":"ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link"}).text
    
    #print(isim.text)
    #print(yapimYili.text)
    #print(sure.text)
    #print(yasSiniri.text)
    #print(puan.text)
    #print(yonetmen.text)
    
    filmler.append([isim, yapimYili, yasSiniri, sure, puan, yonetmen])
    

### Çekilen filmlerin Dataframe de gösterilmesi

In [35]:
df = pd.DataFrame(filmler,columns = ["İsim","Yapım Yılı","Yaş Sınırı","Süre","Puan","Yönetmen"])
df

,İsim,Yapım Yılı,Yaş Sınırı,Süre,Puan,Yönetmen
0,The Shawshank Redemption,1994,13+,2h 22min,9.3,Frank Darabont
1,The Godfather,1972,R,2h 55min,9.2,Francis Ford Coppola
2,The Godfather: Part II,1974,R,3h 22min,9.0,Francis Ford Coppola
3,The Dark Knight,2008,PG-13,2h 32min,9.0,Christopher Nolan
4,12 Angry Men,1957,Approved,1h 36min,9.0,Sidney Lumet
...,...,...,...,...,...,...
245,Sunrise: A Song of Two Humans,1927,Passed,1h 34min,8.1,F.W. Murnau
246,"Paris, Texas",1984,R,2h 25min,8.1,Wim Wenders
247,Le notti di Cabiria,1957,Not Rated,1h 50min,8.1,Federico Fellini
248,Kumonosu-jô,1957,Not Rated,1h 50min,8.1,Akira Kurosawa


### Oluşturulan Dataframe'in excel dosyası olarak kaydedilmesi

In [37]:
df.to_excel('imdb_top250.xlsx')